<a href="https://colab.research.google.com/github/1digitaldesign/sample-code/blob/main/ETL_%26_SQL_Tai%2CBetty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Setup

* Part 1. ETL

* Part 2. SQL 

## Part 1: ETL


In [ ]:
#@title Installation of Necessary Packages 
%%capture
!pip3 install pyspark
!pip3 install hide_code
!pip3 install fancyimpute
!pip3 install miceforest
!pip3 install scikit-learn
%pip install mlxtend --upgrade

On the left, download the dataset [`usa_00001.csv`](https://drive.google.com/file/d/128-eo8QVTYu20aRzA_-7OTxPOjyS-SRp/view?usp=share_link) and on the left hand side, there is a folder to click on called `Files` and then `Upload to Session Storage` this file [`usa_00001.csv`](https://drive.google.com/file/d/128-eo8QVTYu20aRzA_-7OTxPOjyS-SRp/view?usp=share_link). 

In [ ]:
#@title Installation of Necessary Libraries 
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import sqlite3
import requests
import os
import seaborn as sns
import matplotlib.pyplot as plt

## Data Extraction

In [ ]:
df = pd.read_csv('/content/usa_00001.csv')

## Data Transformation
Data transformation is an integral step for data analysis and modeling without errors and improve the performance of the data. 

* Cleaning 
* Normalization
* Reduction
* Discretization
* Aggregation






In [ ]:
df.isnull().sum()
df.shape

In [ ]:
for col_name in df.columns: 
    print(col_name)

In [ ]:
df.describe().style.highlight_max(color="darkred")

In [ ]:
df["FTOTINC"].value_counts()

In [ ]:
df["INCWAGE"].value_counts()

In [ ]:
df_nan = df.copy()
df_nan["FTOTINC"] = df["FTOTINC"].replace(9999999, np.nan)
df_nan["INCWAGE"] = df["INCWAGE"].replace(999999, np.nan)

df_nan.max()

### Imputation of missing data 
There are several techniques we may impute the missing data, and we will examine the bias-variance tradeoff using the [MLxtend](http://rasbt.github.io/mlxtend/) Library. 


*   Substitution by mean
*   Substitution by median
*   Iterative Imputer
*   KNN Imputer




#### Substitution by mean 
I am treating the NaN data as the target variable and doing simple imputation. 

In [ ]:
df_nan.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
train_mean = df_nan[['BIRTHYR','FTOTINC','INCWAGE']].copy()
mean_imputer = SimpleImputer(strategy='mean')
train_mean.iloc[:,:] = mean_imputer.fit_transform(train_mean)
train_mean.isnull().sum()
df_mean = df_nan.fillna(train_mean)
df_mean.isnull().sum()

#### Substitution by median

In [ ]:
from sklearn.impute import SimpleImputer
train_median = df_nan[['BIRTHYR','FTOTINC','INCWAGE']].copy()
median_imputer = SimpleImputer(strategy='median')
train_median.iloc[:,:] = median_imputer.fit_transform(train_median)
train_median.isnull().sum()
df_median = df_nan.fillna(train_median)
df_median.isnull().sum()

#### Iterative Imputer

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

itimp_FTOTINC = pd.DataFrame(np.array(df_nan['FTOTINC']))
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(itimp_FTOTINC)
itimp_FTOTINC = imp_mean.transform(itimp_FTOTINC)
itimp_FTOTINC = pd.DataFrame(itimp_FTOTINC).rename(columns={0:"FTOTINC"})
BIRTHYR_FTOTINC = pd.concat([df_nan['BIRTHYR'], itimp_FTOTINC], axis=1)

In [ ]:
itimp_INCWAGE = pd.DataFrame(np.array(df_nan['INCWAGE']))
imp_mean = IterativeImputer(random_state=0)
imp_mean.fit(itimp_INCWAGE)
itimp_INCWAGE = imp_mean.transform(itimp_INCWAGE)
itimp_INCWAGE = pd.DataFrame(itimp_INCWAGE).rename(columns={0:"INCWAGE"})
BIRTHYR_INCWAGE = pd.concat([df_nan['BIRTHYR'], itimp_INCWAGE], axis=1)
df_iterativeimputer = df_nan.fillna(BIRTHYR_FTOTINC).fillna(BIRTHYR_INCWAGE)

print(f"If there are any nulls in the entire dataframe  : {pd.DataFrame(df_iterativeimputer).isnull().sum()}")

## Part 2: Creating the database & SQL 

In [ ]:
conn = sqlite3.connect('censusData_median.db')
df_median.to_sql('censusDatabaseMedian', con = conn, if_exists='replace', index=False)
pd.read_sql('SELECT * FROM censusDatabaseMedian WHERE AGE > "40"', con = conn).head()

References: 

**Data Set**

Steven Ruggles, Sarah Flood, Matthew Sobek, Danika Brockman, Grace Cooper,  Stephanie Richards, and Megan Schouweiler. IPUMS USA: Version 13.0 [dataset]. Minneapolis, MN: IPUMS, 2023. https://doi.org/10.18128/D010.V13.0